In [2]:
from scipy import interpolate
import numpy as np
import h5py
import pandas as pd
from osgeo import gdal, osr
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## read S2 SRF

In [3]:
df = pd.read_csv('S2-SRF_COPE-GSEG-EOPG.csv')

In [4]:
S2_B= {}
for i in  range (1,len(df.columns)):   
    S2_B[i-1]= list(zip(df.loc[:,df.columns[0]],df.loc[:,df.columns[i]]))
    S2_B[i-1] = {key:val for key, val in list(S2_B[i-1]) if val != 0.0} ## remove all 0 in the value

## loading NEON

In [5]:
f=h5py.File('K:/NEON_refl-surf-dir-ortho-line (3)/NEON_refl-surf-dir-ortho-line/NEON.D16.ABBY.DP1.30006.001.2019-07.basic.20221110T005401Z.RELEASE-2022/NEON_D16_ABBY_DP1_20190714_220751_reflectance.h5','r')

In [6]:
site='ABBY'
S2_Bands=13

In [7]:
Ref = f[site]['Reflectance']
RefArray = Ref['Reflectance_Data']
Ref_shape = RefArray.shape
MapInfo = Ref['Metadata']['Coordinate_System']['Map_Info']
noDataValue =RefArray.attrs['Data_Ignore_Value']
scaleFactor = RefArray.attrs['Scale_Factor']
wavelengths = Ref['Metadata']['Spectral_Data']['Wavelength']
neonwave=wavelengths[()] ## NEON wavelength

In [12]:
neon_S2_B=np.zeros(shape=(13,426))

In [13]:
## S2 13 bands, index from 0 to 12, interpolate S2 SRF for each NEON spectral band
for i in range(13):
    neon_S2_B[i]=np.interp(neonwave,np.fromiter(S2_B[i].keys(), dtype=float), np.fromiter(S2_B[i].values(), dtype=float), 0, 0)

In [14]:
Sim_Output=np.zeros(shape=(Ref_shape[0],Ref_shape[1],S2_Bands))

In [18]:
for j in range(Ref_shape[1]):
    if (j%100==0):
        print ('j=',j)
    temp=RefArray[:,j,:]# get one row of the array to make the process faster
    for i in range(Ref_shape[0]):
        # if (j%100==0):
        #     print ('j=',j)
        # pixel_vec = RefArray[i,j,:].astype(np.float); ## get the pixel vector for data cube
        pixel_vec = temp[i,:].astype(np.float); ## get the pixel vector for data cube
        pixel_vec[pixel_vec==int(noDataValue)]=np.nan; ## assign np.nan to no data 
        pixel_vec=pixel_vec/scaleFactor; ## divide by scalefactor 
        for k in range(S2_Bands):            
            Sim_Output[i][j][k]=np.dot(pixel_vec,neon_S2_B[k])/(np.count_nonzero(neon_S2_B[k])); ### pixel_vec dot production with SRF, divided by the number of bands falling in this band seperacal range.
          

j= 0
j= 100
j= 200
j= 300
j= 400
j= 500
j= 600
j= 700
j= 800
j= 900
j= 1000


In [81]:
def CreateGeoTiff(Name, Array, driver, NDV, 
                  GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    # DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet = driver.Create(Name, Array.shape[1], Array.shape[0], Array.shape[2], DataType)  ## col, row, bands
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection( Projection.ExportToWkt() )
    for i in range(Array.shape[2]):       
        DataSet.GetRasterBand(i+1).WriteArray(Array[:,:,i] )
        DataSet.GetRasterBand(i+1).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

In [72]:
MapInfo_string = str(MapInfo[()]) #convert to string
MapInfo_split = MapInfo_string.split(",") #split the strings using the separator "," 
print(MapInfo_split)
xres = float(MapInfo_split[5])
yres=float(MapInfo_split[6])
print('Resolution:',xres,yres)
#Extract the upper left-hand corner coordinates from mapInfo
xMin = float(MapInfo_split[3]) 
yMax = float(MapInfo_split[4])

["b'UTM", '  1.000', '  1.000', '  552210.000', '  5074055.000', '  1.0000000000e+000', '  1.0000000000e+000', '  10', '  North', '  WGS-84', '  units=Meters', " 0'"]
Resolution: 1.0 1.0


In [73]:
## set the projection information 
srs = osr.SpatialReference()
srs.ImportFromEPSG(32610)   
geotransform=(xMin,xres,0,yMax,0, -yres) 
data_type = gdal.GDT_Float32
driver = gdal.GetDriverByName('GTiff')

In [90]:
CreateGeoTiff('test4.tif', Sim_Output, driver, -9999, geotransform, srs, data_type)

'test4.tif'

In [93]:
## for image resampling from 1m to 10m
infn = 'test4.tif'
outfn = 'test6.tif'
xRes=10
yRes=10

# resample_alg = 'near'
# ds=gdal.Warp(outfn, infn, warpoptions=dict(xRes=xres, yRes=yres, resampleAlg=resample_alg))
# ds = None

# raster_rprj = gdal.Warp(outfn, infn, xRes = 10, yRes = 10, resampleAlg = "near")
raster_rprj = gdal.Warp(outfn, infn, xRes=10, yRes=10, resampleAlg = "near")
raster_rprj = None
# gdal.Warp('outputRaster.tif', 'inputRaster.tif', xRes=10, yRes=10)